<a href="https://colab.research.google.com/github/Somiya11/trial/blob/main/Finetuned_model_on_part2_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 31.3 MB/s eta 0:00:00


In [2]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00


In [8]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from transformers import Trainer, TrainingArguments

In [7]:
class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.questions = df['question'].tolist()
        self.queries = df['sql'].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        query = self.queries[idx]

        inputs = self.tokenizer.encode_plus(
            question,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        labels = self.tokenizer.encode(
            query,
            padding='max_length',
            truncation=True,
            max_length=64,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': labels.squeeze()
        }

In [5]:
df = pd.read_excel('NLtoSQL_dataset_part2.xls')

In [6]:
df

,Natural_language,SQL
0,How many unique major codes are there in oltas?,SELECT DISTINCT(MAJ_HD_CD) FROM Table_oltas
1,How many unique minor codes are there in oltas?,SELECT DISTINCT(MINR_HD_CD) FROM Table_oltas
2,How many unique bank codes are there in oltas?,SELECT DISTINCT(BANK_CD) FROM Table_oltas
3,How many unique branch codes are there in oltas?,SELECT DISTINCT(BRANCH_CD) FROM Table_oltas
4,List all unique taxpayers who have deposited c...,SELECT DISTINCT(TAN_PAN) FROM Table_oltas WHER...
...,...,...
194,"List full name, age and tax paid by taxpayers ...","Select A.Full_Name, B.TAX\nfrom Table_PAN A\nl..."
195,"List full name, age and tax paid by taxpayers ...","Select A.Full_Name, B.TAX\nfrom Table_PAN A\nl..."
196,What are the maximum and minimum tax paid in I...,"SELECT MAX(TAX), MIN(TAX) FROM Table_Itr1"
197,"What are the maximum, minimum and average tax ...","SELECT MAX(TAX), MIN(TAX), AVG(TAX) FROM Table..."


# New Section

# New Section

In [9]:
df=df.sample(frac=1)

In [10]:
df.head()

,Natural_language,SQL
51,"Show the latest filing sequence, Form_id, fili...","SELECT FILING_SEQ_NO,FILING_TYPE,PAN_NO,FORM_I..."
137,show all unique non corporate entities in olta...,SELECT DISTINCT(TAN_PAN) from Table_oltas wher...
67,Show all data in Charity table,SELECT * FROM Table_charity
90,Show minimum TAX in ITR1 table,SELECT Min(TAX) FROM Table_itr1
80,Show all data in ITR3 table where PAN is not i...,SELECT * FROM Table_itr3 where PAN<>'invalid'


In [ ]:
#df=df.head(950)

In [11]:
df2=df.copy()

In [12]:
df2.head()

,Natural_language,SQL
51,"Show the latest filing sequence, Form_id, fili...","SELECT FILING_SEQ_NO,FILING_TYPE,PAN_NO,FORM_I..."
137,show all unique non corporate entities in olta...,SELECT DISTINCT(TAN_PAN) from Table_oltas wher...
67,Show all data in Charity table,SELECT * FROM Table_charity
90,Show minimum TAX in ITR1 table,SELECT Min(TAX) FROM Table_itr1
80,Show all data in ITR3 table where PAN is not i...,SELECT * FROM Table_itr3 where PAN<>'invalid'


In [13]:
df2.rename(columns={'Natural_language':'question', 'SQL':'sql'}, inplace=True)

In [ ]:
#df_new=pd.concat([df, df2])

In [ ]:
#df_new.shape

(1001, 2)

In [14]:
df_new=df2.copy()

In [15]:
train_data = df_new
valid_data = df_new[170:]
#test_data = df[45000:]

In [16]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [17]:
dataset1 = SQLDataset(train_data, tokenizer)
dataset2 = SQLDataset(valid_data, tokenizer)

In [18]:
dataset1[0]

{'input_ids': tensor([ 3111,     8,  1251,  9479,  5932,     6,  3025,   834,    23,    26,
             6,  9479,   833,    11,  9479,   686,    13,   276,  5033, 14213,
           308, 14574,  2128,   948,    16,     3, 15029,  6503,     1,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [19]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=100,
    per_device_train_batch_size=3,    #changed from 16 to 5
    per_device_eval_batch_size=3,
    learning_rate=1e-5,
    save_total_limit=1,
    fp16=True,  # Enable mixed-precision training if available
    logging_dir='./logs',
    logging_steps=10,  # changed from 500 to 10
    save_steps=500,
)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset1,
    eval_dataset=dataset2,
)


In [22]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [23]:
trainer.train()  #training 1

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,12.430500
20,10.266900
30,8.807800
40,8.559100
50,7.472300
60,6.519800
70,5.259500
80,4.916500
90,4.197200
100,3.692600


TrainOutput(global_step=6700, training_loss=0.3019121612519471, metrics={'train_runtime': 1563.5413, 'train_samples_per_second': 12.728, 'train_steps_per_second': 4.285, 'total_flos': 3029565505536000.0, 'train_loss': 0.3019121612519471, 'epoch': 100.0})

In [36]:
trainer.train()  #training 2

Step,Training Loss
10,0.028000
20,0.047900
30,0.055400
40,0.025700
50,0.046900
60,0.041100
70,0.037100
80,0.058300
90,0.039500
100,0.051100


TrainOutput(global_step=6700, training_loss=0.04185098000871602, metrics={'train_runtime': 1565.4349, 'train_samples_per_second': 12.712, 'train_steps_per_second': 4.28, 'total_flos': 3029565505536000.0, 'train_loss': 0.04185098000871602, 'epoch': 100.0})

In [ ]:
save_path = 'FinetunedT5_small_model'
!mkdir {save_path}
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('FinetunedT5_small_model/tokenizer_config.json',
 'FinetunedT5_small_model/special_tokens_map.json',
 'FinetunedT5_small_model/spiece.model',
 'FinetunedT5_small_model/added_tokens.json')

In [ ]:
!zip -r Finetuned_small.zip {save_path}

  adding: FinetunedT5_small_model/ (stored 0%)
  adding: FinetunedT5_small_model/pytorch_model.bin (deflated 9%)
  adding: FinetunedT5_small_model/spiece.model (deflated 48%)
  adding: FinetunedT5_small_model/special_tokens_map.json (deflated 86%)
  adding: FinetunedT5_small_model/tokenizer_config.json (deflated 83%)
  adding: FinetunedT5_small_model/config.json (deflated 62%)
  adding: FinetunedT5_small_model/generation_config.json (deflated 29%)


In [ ]:
output_dir = './fine-tuned-model'
trainer.save_model(output_dir)

In [24]:
results = trainer.evaluate(dataset2)
print(results)

{'eval_loss': 0.013931747525930405, 'eval_runtime': 0.4369, 'eval_samples_per_second': 66.37, 'eval_steps_per_second': 22.886, 'epoch': 100.0}


In [37]:
#training 2
results = trainer.evaluate(dataset2)
print(results)

{'eval_loss': 0.013928608037531376, 'eval_runtime': 0.4193, 'eval_samples_per_second': 69.169, 'eval_steps_per_second': 23.851, 'epoch': 100.0}



**Check the ouput of the model trained twice for 100 epochs each**

In [39]:
input_question = "What is the average total tax paid by coporate entities in year 2023"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT AVG(TAX) FROM Table_oltas WHERE AST_YR=2023</s>


In [42]:
input_question = "What is the minimum regular tax paid by coporate entities in oltas table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MIN(TOT_AMT) FROM Table_oltas where MAJ_HD_CD=20</s>


In [44]:
input_question = "What is the maximum and minimum advance tax paid by coporate entities in oltas table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=100 AND MAJ_HD_CD=20</s>


In [47]:
input_question = "Show all data in ITR5 table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT * FROM Table_itr5</s>


In [48]:
input_question = "Show count of distinct taxpayers in ITR2 table	"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT COUNT(DISTINCT Taxpayers) FROM Table_itr2</s>


In [52]:
input_question = "Show full name and tax paid by PAN='abc123123"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_PAN on A.Pan='abc123123'</s>


In [53]:
input_question = "Show full name and tax paid"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_Tax on A.Pan=B.Pan</s>


In [57]:
input_question = "Show full name, age and tax paid whose Age is 50"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_Pan where Age=50</s>


In [58]:
input_question = "Show full name, age and tax paid whose Age between 30 and 50"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.Full_Name, B.TAX from Table_PAN A left join Table_Tax on A.Pan=B.Pan where Age Between 30 and 50</s>


In [60]:
input_question = "How many unique sft codes are there"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(sft_cd) FROM Table_sft</s>


In [62]:
input_question = "unique sft codes in sft table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(sft_cd) from Table_sft</s>


In [ ]:
input_question = "unique sft codes in sft table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

In [69]:
input_question = "Show maximum tax paid by non corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TAX) FROM Table_oltas WHERE MINR_HD_CD=100 AND MAJ_HD_CD=21 AND AST_YR=21</s>


In [71]:
input_question = "Show full name of taxpayers who were born after 1990 "
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT FULL_NAME from Table_PAN where BIRTH_YEAR>1990</s>


In [79]:
input_question = "Show all personal information and GST paid by taxpayers"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.*, B.TAX from Table_PAN A left join Table_GST on A.Pan=B.Pan</s>


In [80]:
input_question = "Show all personal information and GST paid by taxpayers paid in 2020"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> Select A.*, B.TAX from Table_PAN A left join Table_GST on A.Pan=B.Pan</s>


In [81]:
input_question = "Show all data in GST table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT * FROM Table_GST</s>


In [83]:
input_question = "Show all data in Aman table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT * FROM Table_aman</s>


In [84]:
input_question = "Show max and min amount in table money"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Max(TOT_AMT) FROM Table_money</s>


In [86]:
input_question = "Show maximum, minimum amount in table money"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Max(TOT_AMT) FROM Table_money</s>


In [87]:
input_question ="What are the maximum, minimum and average tax paid in Itr1 table"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TAX), MIN(TAX), AVG(TAX) FROM Table_Itr1</s>


In [88]:
input_question ="List PAN of all unique corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(TAN_PAN) from Table_oltas where MAJ_HD_CD=20</s>


In [89]:
input_question ="List PAN of all unique non corporate entities"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(TAN_PAN) from Table_oltas where MAJ_HD_CD=21</s>


In [92]:
input_question ="Show max self_assessment tax"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=300</s>


In [93]:
input_question ="Show max regular tax "
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Max(TOT_AMT) FROM Table_oltas WHERE MINR_HD_CD=100 AND MAJ_HD_CD=20</s>


In [49]:
#END

** END ** **bold text**

In [63]:
df_new.head(50)

,question,sql
51,"Show the latest filing sequence, Form_id, fili...","SELECT FILING_SEQ_NO,FILING_TYPE,PAN_NO,FORM_I..."
137,show all unique non corporate entities in olta...,SELECT DISTINCT(TAN_PAN) from Table_oltas wher...
67,Show all data in Charity table,SELECT * FROM Table_charity
90,Show minimum TAX in ITR1 table,SELECT Min(TAX) FROM Table_itr1
80,Show all data in ITR3 table where PAN is not i...,SELECT * FROM Table_itr3 where PAN<>'invalid'
1,How many unique minor codes are there in oltas?,SELECT DISTINCT(MINR_HD_CD) FROM Table_oltas
2,How many unique bank codes are there in oltas?,SELECT DISTINCT(BANK_CD) FROM Table_oltas
158,What is the average self_assessment tax in olt...,SELECT AVG(TOT_AMT) FROM Table_oltas WHERE M...
171,Show full name of taxpayers and tax paid in Itr3,"Select A.Full_Name, B.TAX\nfrom Table_PAN A\nl..."
109,Show total tax paid in ITR4 table,SELECT Sum(TAX) FROM Table_itr4


In [ ]:
#generation from newly trained

In [98]:
input_question = "what are unique major codes in oltas"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT DISTINCT(MAJ_HD_CD) FROM Table_oltas</s>
